In [1]:
import pandas as pd

In [2]:
# df_sg = pd.read_parquet('./data/clean/steam_games.parquet.gzip')
df_sg = pd.read_parquet('https://github.com/xaviac/storage__PI_MLOp/raw/main/data/clean/steam_games.parquet.gz')

# df_sg_explode = pd.read_parquet('./data/clean/sg_explode.parquet.gzip')
df_sg_explode = pd.read_parquet('https://github.com/xaviac/storage__PI_MLOp/raw/main/data/clean/sg_explode.parquet.gz')

# df_ur = pd.read_parquet('./data/clean/user_reviews.parquet.gzip')
df_ur = pd.read_parquet('https://github.com/xaviac/storage__PI_MLOp/raw/main/data/clean/user_reviews.parquet.gz')

# df_ui = pd.read_parquet('./data/clean/users_items.parquet.gzip')
df_ui = pd.read_parquet('https://github.com/xaviac/storage__PI_MLOp/raw/main/data/clean/users_items.parquet.gz')

In [3]:
df_sg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32133 entries, 0 to 32132
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        32133 non-null  object 
 1   title         32133 non-null  object 
 2   price         32133 non-null  float64
 3   id            32133 non-null  int32  
 4   developer     32133 non-null  object 
 5   release_year  32133 non-null  int32  
dtypes: float64(1), int32(2), object(3)
memory usage: 1.2+ MB


In [4]:
df_sg.head()

,genres,title,price,id,developer,release_year
0,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,4.99,761140,Kotoshiro,2018
1,"[Free to Play, Indie, RPG, Strategy]",Ironbound,0.00,643980,Secret Level SRL,2018
2,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,0.00,670290,Poolians.com,2017
3,"[Action, Adventure, Casual]",弹炸人2222,0.99,767400,彼岸领域,2017
4,"[Action, Indie, Casual, Sports]",Log Challenge,2.99,773570,Unknown,2015


In [5]:
def developer(desarrollador: str):
    """
    Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora
    params:
    desarrollador: str
    """
    # Verifiquemos que se ha introducido un desarrollador que existe en el dataset
    if desarrollador.title() not in list(df_sg['developer'].str.title()):
        return "Desarrollador no encontrado, por favor, inténtelo de nuevo." 
    
    group_by_year = df_sg[df_sg['developer'].str.title() == desarrollador.title()].groupby('release_year')['id'].count().reset_index()


    # Conteo de 'Free to Play' por año
    free_content = df_sg[(df_sg['developer'] == desarrollador) & (df_sg['price'] == 0)].groupby('release_year')['id'].count().reset_index()
    free_content.rename(columns={'id': 'Free to Play'}, inplace=True)
    
    # Unión de group_by_year y free_content
    merged_data = pd.merge(group_by_year, free_content, on='release_year', how='left')
    
    # Porcentaje del contenido gratis por año
    percent_free = round(merged_data['Free to Play'] / merged_data['id'] * 100, 0)

    list_percent_free = percent_free.fillna(0).tolist()

    # Convertir a cadena y formatear la presentación
    formatted_list = [f"{int(num)}%" for num in list_percent_free]

    
    # Creo el DataFrame final
    resultado = {
        'Año': merged_data['release_year'].tolist(),
        'Cantidad de Items': merged_data['id'].tolist(),
        'Contenido Free': formatted_list
    }
    
    return resultado

In [6]:
developer('Valve')

{'Año': [1998,
  1999,
  2000,
  2001,
  2003,
  2004,
  2005,
  2006,
  2007,
  2008,
  2009,
  2010,
  2011,
  2012,
  2014,
  2016,
  2017],
 'Cantidad de Items': [1, 1, 2, 1, 1, 5, 1, 2, 3, 1, 1, 2, 1, 2, 7, 1, 2],
 'Contenido Free': ['0%',
  '0%',
  '0%',
  '0%',
  '0%',
  '0%',
  '0%',
  '0%',
  '33%',
  '0%',
  '0%',
  '50%',
  '0%',
  '0%',
  '0%',
  '100%',
  '0%']}

In [7]:
df_ur.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             59305 non-null  object
 1   item_id             59305 non-null  int32 
 2   recommend           59305 non-null  bool  
 3   sentiment_analysis  59305 non-null  int64 
dtypes: bool(1), int32(1), int64(1), object(1)
memory usage: 1.2+ MB


In [8]:
df_ui.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3285246 entries, 0 to 3285245
Data columns (total 5 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   user_id                 object 
 1   items_count             int64  
 2   item_id                 int32  
 3   item_name               object 
 4   playtime_forever_hours  float64
dtypes: float64(1), int32(1), int64(1), object(2)
memory usage: 112.8+ MB


In [9]:
# print(df_ui['user_id'].nunique())
# Imprimir los valores únicos de la columna 'user_id'
print(df_ui['user_id'].unique().tolist())
print(len(df_ui['user_id'].unique().tolist()))

['76561197970982479', 'js41637', 'evcentric', 'Riot-Punch', 'doctr', 'MinxIsBetterThanPotatoes', 'NitemarePK', 'themanwich', 'maplemage', 'corrupted_soul', 'jorellpogi', 'cadmusthreepointoh', '76561198064823719', 'MeaTCompany', '76561198089393905', '76561198156664158', '76561198077246154', '76561198070234207', 'WeiEDKrSat', 'thequeenpanda', 'death-hunter', 'turneddreamsintoanempire', 'DJKamBer', '76561198030389591', 'Rainbow-Dashie', 'btuty8trgt', '76561198043472122', 'MarbleShrine', 'fdcfelipefdc', 'bbaekhyun', 'Moomfina', 'PPanther', 'ABSOLUTLY_NOTHING', 'devvonst', 'Fr0stedLine', '76561198058373434', '76561198048353577', '76561198072190369', '76561198066046412', 'tru_mu_', 'wirlom', '76561198070565427', 'SavageEeEe', '76561198076909484', '76561198066908262', 'KanadeTheNeko', 'Leaf_Light_Moscow', 'erickoiv', 'GoAwayZach', '76561198054540475', '76561198100701221', '76561198085547365', '76561198061252210', 'OfficialShroomsy', '76561198056147391', '8Bit-Ninja', '76561198071500702', '765

In [10]:
# Hago un merge de  los dataframes 'df_sg' y 'df_ui' en 'union_sg_ui_5_columns' basándonos en 'title' y 'item_name'
union_sg_ui_5_columns = pd.merge(df_sg[['title','price']],df_ui[['user_id','items_count','item_name']],left_on='title',right_on='item_name',how='inner')

# Agrupo los datos en 'users_group' por 'user_id' y agregamos la primera aparición de 'items_count' y la suma de 'price'
users_group = union_sg_ui_5_columns.groupby('user_id').agg(
    {'items_count':'first',
     'price':'sum'}
     ).reset_index()

# Redondear la columna 'price' a 2 decimales
users_group['price'] = users_group['price'].round(2)

# Sumamos las recomendaciones por usuario en 'recommend_true'
recommend_true = df_ur.groupby('user_id')['recommend'].sum()

# Realizar el agrupamiento
total_items_user = df_ui.groupby('user_id')['item_id'].count()

# Calculamos el porcentaje de recomendaciones en 'percent_recommend'
percent_recommend = round((recommend_true / total_items_user) * 100, 2)

# Nuevo dataframe 'users_true_reviews' con 'user_id' y 'percent_recommend'
users_true_reviews = pd.DataFrame({
    'user_id': percent_recommend.index,
    'percent_recommend': percent_recommend.values})

# Unir total_items_user y users_true_reviews por el id    
users_true_reviews = pd.merge(users_true_reviews,total_items_user,on='user_id',how='left')

# Renombrar 'item_id' como 'total_items'
users_true_reviews.rename(columns={'item_id':'total_items'},inplace=True)

# Uno 'users_group' y 'users_true_reviews' en 'items_reviews_users'
items_reviews_users = pd.merge(users_group,users_true_reviews,on='user_id',how='left')

In [11]:
items_reviews_users.to_csv('../data/functions/items_reviews_users.csv.gz', compression='gzip',index=False, encoding='utf-8')

items_reviews_users.to_parquet('../data/functions/items_reviews_users.parquet.gz', compression='gzip',index=False)

In [12]:
def userdata(User_id:str):
    """
    Retorna el dinero gastado, el porcentaje de recomendación y la cantidad de items comprados por el usuario
    params:
    User_id: str
    """
    # Se genera un dataframe para el User_id ingresado de donde se leerán los datos.
    user = items_reviews_users[items_reviews_users['user_id'] == User_id]
    if user.empty:
        return "Usuario no encontrado"
    
    return {
        'Usuario': str(user['user_id'].iloc[0]),
        'Dinero gastado': f'{str(user["price"].iloc[0])} USD',
        '% de recomendación': f'{str(user['percent_recommend'].iloc[0])}%',
        'Cantidad de items': str(int(user['total_items'].iloc[0]))
        }

In [13]:
userdata('chidvd')

{'Usuario': 'chidvd',
 'Dinero gastado': '21363.0 USD',
 '% de recomendación': '0.23%',
 'Cantidad de items': '3475'}

In [24]:
union_ui_sg = pd.merge(df_ui, df_sg_explode, left_on='item_name', right_on='title', how='inner')

In [25]:
group_by_user_genres_year = pd.DataFrame(union_ui_sg.groupby(['user_id', 'genres', 'release_year'])['playtime_forever_hours'].sum().reset_index())

In [26]:
# Eliminar valores < 0 de playtime_forever_hours
group_by_user_genres_year = group_by_user_genres_year[group_by_user_genres_year['playtime_forever_hours'] > 1]

In [ ]:
# Guardar el dataframe en un archivo csv en la carpeta functions
# group_by_user_genres_year.to_csv('../data/functions/group_by_user_genres_year.csv.gz', compression='gzip', index=False, encoding='utf-8')

In [ ]:
# Guardar el dataframe en un archivo parquet en la carpeta functions
# group_by_user_genres_year.to_parquet('../data/functions/group_by_user_genres_year.parquet.gz', compression='gzip', index=False)

In [22]:
def UserForGenre(genero: str):
    # Filtrar el DataFrame por el género dado

    generes = group_by_user_genres_year[group_by_user_genres_year['genres'].str.contains(genero)] 
    #Agrupo por usuario y sumo cantidad de horas jugadas
    hours = generes.groupby('user_id')['playtime_forever_hours'].sum().reset_index()


    # Encontrar el usuario con más horas jugadas para ese género
    playtime_by_user = hours.loc[hours['playtime_forever_hours'].idxmax()]['user_id']


    # Filtrar el DataFrame por el usuario con más horas jugadas en ese género
    top_user = generes[generes['user_id'] == playtime_by_user]
    total_hours_year = top_user.groupby('release_year')['playtime_forever_hours'].sum().reset_index()

    user_hours_years = [{'Año': int(row['release_year']), 'Horas': int(row['playtime_forever_hours'])} for _, row in total_hours_year.iterrows()]
    
    return {f'Usuario con más horas jugadas para {genero}': playtime_by_user, 'Horas jugadas': user_hours_years}

In [27]:
UserForGenre('Action')

{'Usuario con más horas jugadas para Action': 'shinomegami',
 'Horas jugadas': [{'Año': 1988, 'Horas': 2},
  {'Año': 1996, 'Horas': 19},
  {'Año': 1997, 'Horas': 1},
  {'Año': 1998, 'Horas': 61},
  {'Año': 1999, 'Horas': 5},
  {'Año': 2000, 'Horas': 179},
  {'Año': 2001, 'Horas': 40},
  {'Año': 2003, 'Horas': 8879},
  {'Año': 2004, 'Horas': 2},
  {'Año': 2005, 'Horas': 5},
  {'Año': 2006, 'Horas': 34},
  {'Año': 2007, 'Horas': 1139},
  {'Año': 2008, 'Horas': 3},
  {'Año': 2009, 'Horas': 3496},
  {'Año': 2010, 'Horas': 156},
  {'Año': 2011, 'Horas': 1489},
  {'Año': 2012, 'Horas': 308},
  {'Año': 2013, 'Horas': 2469},
  {'Año': 2014, 'Horas': 1534},
  {'Año': 2015, 'Horas': 3033},
  {'Año': 2016, 'Horas': 709},
  {'Año': 2017, 'Horas': 2}]}

In [14]:
union_ur_sg = pd.merge(df_ur, df_sg, left_on='item_id', right_on='id', how='inner')

In [15]:
union_ur_sg = union_ur_sg.drop(columns=['item_id', 'id','genres'], axis=1)

In [16]:
# Recorrer 'title' buscando '&amp;' verificando que fue corregido
union_ur_sg['title'][union_ur_sg['title'].apply(lambda x: '&amp;' in x)]

Series([], Name: title, dtype: object)

In [17]:
# Recorrer con un for y sustituir dentro de la columna title '&amp;' por '&'
for title in union_ur_sg['title'][union_ur_sg['title'].apply(lambda x: '&amp;' in x)]:
    if isinstance(title, str):
        union_ur_sg['title'].replace(title, title.replace('&amp;', '&'), inplace=True)

In [18]:
# Recorrer 'developer' buscando '&amp;' verificando que fue corregido
union_ur_sg['developer'][union_ur_sg['developer'].apply(lambda x: '&amp;' in x)]

Series([], Name: developer, dtype: object)

In [19]:
# Recorrer con un for y sustituir dentro de la columna developer '&amp;' por '&'
for developer in union_ur_sg['developer'][union_ur_sg['developer'].apply(lambda x: '&amp;' in x)]:
    if isinstance(developer, str):
        union_ur_sg['developer'].replace(developer, developer.replace('&amp;', '&'), inplace=True)

In [20]:
def best_developer_year(year: int):
    # Filtramos el DataFrame donde la columna 'release_year' es igual a year, la columna 'recommend' es True y la columna 'sentiment_analysis' tiene valores 2 positivos. 
    filter = union_ur_sg['developer'][(union_ur_sg['release_year'] == year) & (union_ur_sg['recommend'] == True) & (union_ur_sg['sentiment_analysis'].isin([2]))].value_counts().reset_index().head(3)
    
    return [{f'Puesto {i + 1}: {row['developer']}'} for i, row in filter.iterrows()]

In [21]:
best_developer_year(2000)

[{'Puesto 1: Valve'}, {'Puesto 2: Ion Storm'}, {'Puesto 3: Rebellion'}]

In [30]:
def developer_reviews_analysis(developer: str):
    # Filtrar el DataFrame para el año
    filter = union_ur_sg[union_ur_sg['developer'] == developer]
    #Cuenta los comentarios positivos
    positives = filter[filter['sentiment_analysis']==2]['sentiment_analysis'].count()
    # Cuenta los comentarios negativos
    negatives = filter[filter['sentiment_analysis']==0]['sentiment_analysis'].count()
    # Cuenta los comentarios neutrales
    neutrals = filter[filter['sentiment_analysis']==1]['sentiment_analysis'].count()

    # Devolver conteos en un diccionario con el nombre del desarrollador como clave
    # y una lista de los conteos como valor negativos y positivos
    return {f'{developer}':[f'Negative = {negatives}', f'Positive = {positives}']}

In [31]:
developer_reviews_analysis('Valve')

{'Valve': ['Negative = 559', 'Positive = 4447']}